# Horizon Catalog - UI Walkthrough
**Understood for All - Hands On Lab**

---

## What is Horizon Catalog?

Horizon Catalog is Snowflake's **data discovery and governance** layer. Think of it as a GPS for your data:
- Find any table, view, or object instantly
- See where data comes from (lineage)
- Understand what data means (tags, descriptions)
- Track who's using what

**This demo is 100% UI-based** - no code needed, just navigation!

---

**Prerequisites:** Run the Dynamic Tables demo first to have objects with lineage to show.

---
## Part 1: Finding Your Data

### Step-by-step:

1. **Open Snowsight** (the Snowflake web UI)

2. **Click "Data"** in the left sidebar

3. **Click "Databases"**

4. **Expand** `UNDERSTOOD_DEMO` → `DYNAMIC_TABLES`

You should see:
- **Tables:** `cust_info`, `prod_stock_inv`, `salesdata`, `donations`
- **Dynamic Tables:** `customer_sales_data_history`, `salesreport`, `donation_summary`

---

**Key Point:** Everything is browsable and searchable - no SQL needed to explore your data!

---
## Part 2: Exploring Object Details

### Step-by-step:

1. **Click on** `salesreport` (the Dynamic Table)

2. **Look at the tabs:**

| Tab | What It Shows |
|-----|---------------|
| **Details** | Row count, size, when created, who owns it |
| **Columns** | All columns with data types |
| **Data Preview** | Sample rows WITHOUT writing SQL |
| **Graph** | Where the data comes from (lineage) |

3. **Click "Data Preview"** to see sample data instantly

---

**Key Point:** You can understand any table's structure and contents without writing a single query!

---
## Part 3: Data Lineage - The Magic

This is the most powerful feature of Horizon Catalog!

### Step-by-step:

1. **Click on** `salesreport` Dynamic Table

2. **Click the "Graph" tab**

3. **See the lineage visualization:**

```
┌─────────────┐     ┌─────────────┐
│  cust_info  │     │  salesdata  │
└──────┬──────┘     └──────┬──────┘
       │                   │
       └─────────┬─────────┘
                 │
                 ▼
┌─────────────────────────────────┐
│  customer_sales_data_history    │
└────────────────┬────────────────┘
                 │
                 ▼
┌─────────────────────────────────┐     ┌─────────────────┐
│          salesreport            │◄────│  prod_stock_inv │
└─────────────────────────────────┘     └─────────────────┘
```

4. **Click on any node** to see details about that object

5. **Click "Upstream"** or **"Downstream"** to navigate the data flow

---

**Key Points:**
- Snowflake tracks this **automatically** - you did nothing special!
- Every join, transformation, and dependency is captured
- This is critical for compliance: "Where did this number come from?"

---
## Part 4: Tags and Classification

Tags help you organize and govern your data.

### Step-by-step:

1. **Click on any table** (e.g., `cust_info`)

2. **Click the "Columns" tab**

3. **Notice each column shows:**
   - Data type (VARCHAR, NUMBER, DATE)
   - Nullable or not
   - Tags (if applied)

4. **To add a tag:**
   - Click the "..." menu next to a column
   - Select "Add Tag"
   - Choose from existing tags or create new ones

### Common tag categories:
| Tag Type | Example | Purpose |
|----------|---------|---------|
| **PII** | `contains_pii` | Mark columns with personal data |
| **Sensitivity** | `confidential`, `public` | Data classification |
| **Business** | `financial`, `customer_data` | Organizational context |

---

**Key Point:** Tags power governance policies - tag a column as PII, and masking policies can auto-apply!

---
## Part 5: Adding Descriptions

Make your data self-documenting!

### Step-by-step:

1. **Click on** `salesreport`

2. **Click "Details" tab**

3. **Click the edit icon** next to "Description"

4. **Add a description** like:
   > "Final sales report combining customer purchases with product details. Refreshes every minute."

5. **Click Save**

### You can also add descriptions to:
- Individual columns (what does "saleprice" mean?)
- Schemas (what's this schema for?)
- Databases (who owns this?)

---

**Key Point:** Future you (and your teammates) will thank you for documentation!

---
## Part 6: Search and Discovery

Find anything in your data warehouse instantly.

### Step-by-step:

1. **Click the search bar** at the top of Snowsight (or press `/`)

2. **Type** `sales`

3. **See results showing:**
   - Tables matching "sales"
   - Columns matching "sales"
   - Warehouses, roles, users

4. **Use filters** to narrow down:
   - By object type (Tables, Views, Dynamic Tables)
   - By database or schema
   - By owner

---

**Key Point:** No more asking "where is that table?" - just search!

---
## Bonus: What's Coming with Select Star

Snowflake acquired **Select Star** to enhance Horizon Catalog even further.

### Coming soon:
| Feature | What It Does |
|---------|--------------|
| **Data Popularity** | See which tables/columns are actually used vs. stale |
| **Column-Level Lineage** | Track data from source → transformations → dashboards |
| **BI Tool Integration** | See lineage into Tableau, Looker, Power BI |
| **Cross-Platform Visibility** | Track data beyond just Snowflake |

### Why this matters for Understood:
- Know which reports are actually being used
- Trace a number from a dashboard back to its source
- Identify unused tables you can archive
- Complete audit trail for compliance

---

**Key Point:** Horizon + Select Star = Complete visibility across your entire data stack!

---
## Hands-On: Data Protection Policies

Now let's see Horizon's governance features in action with live SQL demos.

In [ ]:
-- Setup: Use our demo schema
USE DATABASE UNDERSTOOD_DEMO;
USE SCHEMA AI_DEMO;

### 1. Dynamic Data Masking

**What it does:** Hides sensitive data from unauthorized users while showing it to authorized ones.

**How it works:** Create a masking policy that returns masked values based on the user's role.

In [ ]:
-- Dynamic Masking Demo: See how data appears to different roles

-- First, view the raw data (you're ACCOUNTADMIN, so you see everything)
SELECT 'As ACCOUNTADMIN - Full Access:' AS view_type;
SELECT family_name, primary_contact_email, phone FROM families LIMIT 3;

-- Now simulate what a restricted role would see
SELECT 'As ANALYST role - Masked View:' AS view_type;
SELECT 
    family_name,
    '***@***.com' AS primary_contact_email,  -- This is what masking would show
    phone
FROM families LIMIT 3;

### 2. Conditional Masking

**What it does:** Masks data based on OTHER column values in the same row.

**How it works:** The policy receives multiple columns and decides masking based on conditions.

In [ ]:
-- Conditional Masking Demo: Mask based on another column's value
-- Example: Only show phone if family gave marketing consent

-- Simulate a consent column and show how masking would work
SELECT 
    family_name,
    phone AS actual_phone,
    CASE subscription_tier
        WHEN 'Premium' THEN phone              -- Premium = consented, show phone
        ELSE '###-###-####'                     -- Others = masked
    END AS conditional_phone,
    subscription_tier AS "consent_proxy"
FROM families 
LIMIT 5;

### 3. Tag-Based Masking

**What it does:** Automatically applies masking to ANY column with a specific tag - no manual policy assignment needed.

**How it works:** Associate a masking policy with a tag. When you tag a column, the policy applies automatically.

In [ ]:
-- Tag-Based Masking Demo: One policy protects ALL tagged columns automatically

-- Show multiple PII columns that would ALL be protected with one tag
SELECT 
    family_id,
    family_name,
    -- These columns would be tagged as PII and auto-masked:
    primary_contact_email AS "email (PII)",
    phone AS "phone (PII)",
    city || ', ' || state AS "location (not PII)"
FROM families LIMIT 3;

-- With tag-based masking, a non-admin would see:
SELECT 
    family_id,
    family_name,
    '**** MASKED ****' AS "email (PII - masked)",
    '**** MASKED ****' AS "phone (PII - masked)", 
    city || ', ' || state AS "location (not PII)"
FROM families LIMIT 3;

### 4. Row Access Policies

**What it does:** Controls which ROWS a user can see based on their role or attributes.

**How it works:** A policy function returns TRUE/FALSE for each row - users only see rows where the policy returns TRUE.

In [ ]:
-- Row Access Policy Demo: Filter which ROWS users can see

-- Full data (as admin, you see all states)
SELECT 'ACCOUNTADMIN sees ALL regions:' AS access_level;
SELECT family_name, city, state FROM families ORDER BY state LIMIT 10;

-- Simulate regional access - a Texas-based user only sees TX families
SELECT 'Regional User (TX) sees only their region:' AS access_level;
SELECT family_name, city, state 
FROM families 
WHERE state = 'TX'  -- Row access policy would filter this automatically
ORDER BY family_name;

### Quick Reference: When to Use Each

| Policy Type | Use Case | Example |
|-------------|----------|---------|
| **Dynamic Masking** | Hide columns from roles | Analysts can't see SSNs |
| **Conditional Masking** | Mask based on row data | Show phone only if consent=TRUE |
| **Tag-Based Masking** | Scale across many tables | All 'PII' tagged columns masked |
| **Row Access Policy** | Limit which rows users see | Specialists see only their families |

---
## Summary

### What we covered:

| Feature | Business Value |
|---------|----------------|
| **Data Browser** | Find any table instantly, no SQL needed |
| **Data Preview** | See sample data without queries |
| **Lineage** | Know where data comes from and where it goes |
| **Tags** | Classify and govern sensitive data |
| **Descriptions** | Self-documenting data warehouse |
| **Search** | Universal search across all objects |

### Key takeaways:

1. **Horizon is your GPS for data** - no more hunting for tables

2. **Lineage is automatic** - Snowflake tracks it without you doing anything

3. **Governance is built-in** - tags, classifications, and policies work together

4. **AI-ready** - good metadata makes AI tools more effective

---

**For Understood:** This foundation enables data governance, compliance reporting, and confident AI adoption!